###**Personalized Medical Recommendation System**

In [1]:
!pip install kaggle

#####**Mounting Google Drive in Google Colab**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Creating a Kaggle Folder
!mkdir ~/.kaggle

In [4]:
!kaggle datasets download -d noorsaeed/medicine-recommendation-system-dataset

Dataset URL: https://www.kaggle.com/datasets/noorsaeed/medicine-recommendation-system-dataset
License(s): apache-2.0
100% 59.8k/59.8k [00:00<00:00, 554kB/s]
100% 59.8k/59.8k [00:00<00:00, 552kB/s]


In [5]:
!unzip medicine-recommendation-system-dataset.zip

Archive:  medicine-recommendation-system-dataset.zip
  inflating: Symptom-severity.csv    
  inflating: Training.csv            
  inflating: description.csv         
  inflating: diets.csv               
  inflating: medications.csv         
  inflating: precautions_df.csv      
  inflating: symtoms_df.csv          
  inflating: workout_df.csv          


#####**Loading Dataset and Tools**

In [6]:
import pandas as pd

In [7]:
dataset=pd.read_csv("/content/TrainingFinal.csv")

In [8]:
dataset.head()

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
1,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
4,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection


In [9]:
dataset.shape

(4925, 133)

In [10]:
dataset['prognosis'].nunique()

42

In [11]:
dataset['prognosis'].unique()

array(['Fungal infection', 'Allergy', 'GERD', 'Chronic cholestasis',
       'Drug Reaction', 'Peptic ulcer diseae', 'AIDS', 'Influenzer (Flu)',
       'Diabetes ', 'Gastroenteritis', 'Bronchial Asthma',
       'Hypertension ', 'Migraine', 'Cervical spondylosis',
       'Paralysis (brain hemorrhage)', 'Jaundice', 'Malaria',
       'Chicken pox', 'Dengue', 'Typhoid', 'hepatitis A', 'Hepatitis B',
       'Hepatitis C', 'Hepatitis D', 'Hepatitis E', 'Alcoholic hepatitis',
       'Tuberculosis', 'Common Cold', 'Pneumonia',
       'Dimorphic hemmorhoids(piles)', 'Heart attack', 'Varicose veins',
       'Hypothyroidism', 'Hyperthyroidism', 'Hypoglycemia',
       'Osteoarthristis', 'Arthritis',
       '(vertigo) Paroymsal  Positional Vertigo', 'Acne',
       'Urinary tract infection', 'Psoriasis', 'Impetigo'], dtype=object)

In [12]:
dataset.isnull().sum()

,0
itching,0
skin_rash,0
nodal_skin_eruptions,0
continuous_sneezing,0
shivering,0
...,...
inflammatory_nails,0
blister,0
red_sore_around_nose,0
yellow_crust_ooze,0


#####**Train Test Split**

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [14]:
X=dataset.drop('prognosis',axis=1)
y=dataset['prognosis']

In [15]:
le=LabelEncoder()
le.fit(y)
Y=le.transform(y)

In [16]:
Y

array([15, 15, 15, ..., 10, 10, 10])

In [17]:
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.3,random_state=20)

In [18]:
X_train.shape,X_test.shape,y_test.shape,y_train.shape

((3447, 132), (1478, 132), (1478,), (3447,))

#####**Training Top Models**

In [19]:
import pandas as pd
from sklearn.datasets import make_classification  # For creating a synthetic dataset
from sklearn.svm import SVC  # Support Vector Classifier
from sklearn.linear_model import LogisticRegression  # Logistic Regression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier # Ensemble methods
from sklearn.neighbors import KNeighborsClassifier  # K-Nearest Neighbors
from sklearn.naive_bayes import MultinomialNB  # Naive Bayes classifier
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold,GridSearchCV   #Cross Validation
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.tree import DecisionTreeClassifier  # Decision Tree Classifier
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [20]:
# Define models with modified parameters for regularization
models = {
    'LogisticRegression': LogisticRegression(C=0.5, solver='liblinear', random_state=42,class_weight="balanced"),  # Logistic Regression
    'SVC': SVC(kernel="linear", C=0.1),  # Lower regularization strength (C=0.1)
    'RandomForest': RandomForestClassifier(n_estimators=50, max_depth=5, random_state=42),  # Tuned RF
    "DecisionTree": DecisionTreeClassifier(max_depth=5, criterion="entropy", random_state=42),
    'GradientBoosting': GradientBoostingClassifier(n_estimators=50, learning_rate=0.05, random_state=42),  # Tuned GB
    'KNeighbors': KNeighborsClassifier(n_neighbors=15, weights='uniform'),  # Regularized KNN (increased neighbors)
    'MultinomialNB': MultinomialNB(alpha=0.5),  # Tuned Naive Bayes
}

# Define Stratified KFold cross-validation strategy to maintain class distribution
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Iterate over each model
for model_name, model in models.items():
    print(f"\nModel: {model_name}")

    # Performed Stratified Cross-validation to evaluate generalization
    cv_scores = cross_val_score(model, X_train, y_train, cv=skf)
    print(f"Cross-Validation Accuracy: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")

    # Trained the model on the full training set
    model.fit(X_train, y_train)

    # Evaluated on training data to check for overfitting
    train_predictions = model.predict(X_train)
    train_accuracy = accuracy_score(y_train, train_predictions)

    # Evaluated on test data
    test_predictions = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, test_predictions)

    # Generated classification report and summarize key metrics
    report = classification_report(y_test, test_predictions, output_dict=True)
    precision = report["weighted avg"]["precision"]
    recall = report["weighted avg"]["recall"]
    f1_score = report["weighted avg"]["f1-score"]

    # Calculated confusion matrix
    cm = confusion_matrix(y_test, test_predictions)

    # Print model evaluation results
    print(f"Training Accuracy: {train_accuracy:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print("Classification Report Summary:")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall: {recall:.4f}")
    print(f"  F1-Score: {f1_score:.4f}")
    print("Confusion Matrix:")
    print(cm)
    print("\n")



Model: LogisticRegression
Cross-Validation Accuracy: 0.9997 ± 0.0009
Training Accuracy: 0.9997
Test Accuracy: 1.0000
Classification Report Summary:
  Precision: 1.0000
  Recall: 1.0000
  F1-Score: 1.0000
Confusion Matrix:
[[36  0  0 ...  0  0  0]
 [ 0  1  0 ...  0  0  0]
 [ 0  0 22 ...  0  0  0]
 ...
 [ 0  0  0 ... 33  0  0]
 [ 0  0  0 ...  0 41  0]
 [ 0  0  0 ...  0  0 30]]



Model: SVC
Cross-Validation Accuracy: 0.9991 ± 0.0013
Training Accuracy: 0.9991
Test Accuracy: 0.9993
Classification Report Summary:
  Precision: 0.9987
  Recall: 0.9993
  F1-Score: 0.9990
Confusion Matrix:
[[36  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0 22 ...  0  0  0]
 ...
 [ 0  0  0 ... 33  0  0]
 [ 0  0  0 ...  0 41  0]
 [ 0  0  0 ...  0  0 30]]



Model: RandomForest
Cross-Validation Accuracy: 0.9571 ± 0.0306
Training Accuracy: 0.9681
Test Accuracy: 0.9567
Classification Report Summary:
  Precision: 0.9507
  Recall: 0.9567
  F1-Score: 0.9497
Confusion Matrix:
[[36  0  0 ...  0  0  0]
 [ 0  0  0 

#####**Single Prediction SVC (Support Vector Classifier)**

In [21]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Define the model
svc = SVC(kernel="linear", C=0.1)

# Set up parameter grid for GridSearchCV
param_grid = {
    'kernel': ['linear', 'rbf', 'poly'],
    'C': [0.1, 1, 10],
    'gamma': ['scale', 'auto']
}

# Initialize GridSearchCV with cross-validation
grid_search = GridSearchCV(svc, param_grid, cv=5, scoring='accuracy', verbose=1)

# Fit GridSearchCV to the training data
grid_search.fit(X_train, y_train)

# Get the best parameters and best estimator
best_params = grid_search.best_params_
print(f"Best Parameters: {best_params}")

# Use the best estimator from GridSearchCV
best_svc = grid_search.best_estimator_

# Define StratifiedKFold for cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Evaluate with cross-validation using StratifiedKFold
cv_scores = cross_val_score(best_svc, X_train, y_train, cv=skf)
print(f"Cross-Validation Accuracy: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")

# Fit the best model to the training data
best_svc.fit(X_train, y_train)

# Make predictions on the test data
y_pred = best_svc.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Generate classification report
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)


Fitting 5 folds for each of 18 candidates, totalling 90 fits
Best Parameters: {'C': 10, 'gamma': 'scale', 'kernel': 'linear'}
Cross-Validation Accuracy: 0.9994 ± 0.0007
Accuracy: 1.0000
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       1.00      1.00      1.00         1
           2       1.00      1.00      1.00        22
           3       1.00      1.00      1.00        34
           4       1.00      1.00      1.00        36
           5       1.00      1.00      1.00        30
           6       1.00      1.00      1.00        32
           7       1.00      1.00      1.00        41
           8       1.00      1.00      1.00        33
           9       1.00      1.00      1.00        39
          10       1.00      1.00      1.00        35
          11       1.00      1.00      1.00        40
          12       1.00      1.00      1.00        38
          13       1.00      1.00 

In [25]:
#Saving Model
import pickle
pickle.dump(best_svc,open("/content/models/svc.pkl",'wb'))

In [26]:
#Load Model
best_svc=pickle.load(open("/content/models/svc.pkl",'rb'))

In [27]:
# Test 1
print("Predicted Label:", best_svc.predict(X_test.iloc[0].values.reshape(1,-1)))
print("Actual Label:", y_test[0])

Predicted Label: [32]
Actual Label: 32


In [28]:
# Test 2
print("Predicted Label:",best_svc.predict(X_test.iloc[10].values.reshape(1,-1)))
print("Actual Label:",y_test[10])

Predicted Label: [23]
Actual Label: 23


In [29]:
#Test 3
print("Predicted Label:",best_svc.predict(X_test.iloc[15].values.reshape(1,-1)))
print("Actual Label:",y_test[15])

Predicted Label: [12]
Actual Label: 12


##**Recommendation System and Prediction**

#####**Load Datasets and used logic for recommendations**

In [30]:
sym_des=pd.read_csv("/content/symptoms_df.csv")
precautions=pd.read_csv("/content/precautions_df.csv")
health_advice=pd.read_csv("/content/workout_df.csv")
description=pd.read_csv("/content/description.csv")
medications=pd.read_csv("/content/medications.csv")
diets=pd.read_csv("/content/diets.csv")

In [31]:
#=================== Custom helper Function ===================
def helper(dis):
  desc=description[description['Disease']==dis]['Description']
  desc=" ".join([w for w in desc])

  pre=precautions[precautions['Disease']==dis][['Precaution_1','Precaution_2','Precaution_3','Precaution_4']]
  pre=[pre for pre in pre.values]

  med=medications[medications['Disease']==dis]['Medication']
  med=[med for med in med.values]

  health=health_advice[health_advice['disease']==dis]['workout']

  diet=diets[diets['Disease']==dis]['Diet']
  diet=[diet for diet in diet.values]

  return desc,pre,med,health,diet

symptoms_dict = {'itching': 0, 'skin_rash': 1, 'nodal_skin_eruptions': 2, 'continuous_sneezing': 3, 'shivering': 4, 'chills': 5, 'joint_pain': 6, 'stomach_pain': 7, 'acidity': 8, 'ulcers_on_tongue': 9, 'muscle_wasting': 10, 'vomiting': 11, 'burning_micturition': 12, 'spotting_ urination': 13, 'fatigue': 14, 'weight_gain': 15, 'anxiety': 16, 'cold_hands_and_feets': 17, 'mood_swings': 18, 'weight_loss': 19, 'restlessness': 20, 'lethargy': 21, 'patches_in_throat': 22, 'irregular_sugar_level': 23, 'cough': 24, 'high_fever': 25, 'sunken_eyes': 26, 'breathlessness': 27, 'sweating': 28, 'dehydration': 29, 'indigestion': 30, 'headache': 31, 'yellowish_skin': 32, 'dark_urine': 33, 'nausea': 34, 'loss_of_appetite': 35, 'pain_behind_the_eyes': 36, 'back_pain': 37, 'constipation': 38, 'abdominal_pain': 39, 'diarrhoea': 40, 'mild_fever': 41, 'yellow_urine': 42, 'yellowing_of_eyes': 43, 'acute_liver_failure': 44, 'fluid_overload': 45, 'swelling_of_stomach': 46, 'swelled_lymph_nodes': 47, 'malaise': 48, 'blurred_and_distorted_vision': 49, 'phlegm': 50, 'throat_irritation': 51, 'redness_of_eyes': 52, 'sinus_pressure': 53, 'runny_nose': 54, 'congestion': 55, 'chest_pain': 56, 'weakness_in_limbs': 57, 'fast_heart_rate': 58, 'pain_during_bowel_movements': 59, 'pain_in_anal_region': 60, 'bloody_stool': 61, 'irritation_in_anus': 62, 'neck_pain': 63, 'dizziness': 64, 'cramps': 65, 'bruising': 66, 'obesity': 67, 'swollen_legs': 68, 'swollen_blood_vessels': 69, 'puffy_face_and_eyes': 70, 'enlarged_thyroid': 71, 'brittle_nails': 72, 'swollen_extremeties': 73, 'excessive_hunger': 74, 'extra_marital_contacts': 75, 'drying_and_tingling_lips': 76, 'slurred_speech': 77, 'knee_pain': 78, 'hip_joint_pain': 79, 'muscle_weakness': 80, 'stiff_neck': 81, 'swelling_joints': 82, 'movement_stiffness': 83, 'spinning_movements': 84, 'loss_of_balance': 85, 'unsteadiness': 86, 'weakness_of_one_body_side': 87, 'loss_of_smell': 88, 'bladder_discomfort': 89, 'foul_smell_of urine': 90, 'continuous_feel_of_urine': 91, 'passage_of_gases': 92, 'internal_itching': 93, 'toxic_look_(typhos)': 94, 'depression': 95, 'irritability': 96, 'muscle_pain': 97, 'altered_sensorium': 98, 'red_spots_over_body': 99, 'belly_pain': 100, 'abnormal_menstruation': 101, 'dischromic _patches': 102, 'watering_from_eyes': 103, 'increased_appetite': 104, 'polyuria': 105, 'family_history': 106, 'mucoid_sputum': 107, 'rusty_sputum': 108, 'lack_of_concentration': 109, 'visual_disturbances': 110, 'receiving_blood_transfusion': 111, 'receiving_unsterile_injections': 112, 'coma': 113, 'stomach_bleeding': 114, 'distention_of_abdomen': 115, 'history_of_alcohol_consumption': 116, 'fluid_overload.1': 117, 'blood_in_sputum': 118, 'prominent_veins_on_calf': 119, 'palpitations': 120, 'painful_walking': 121, 'pus_filled_pimples': 122, 'blackheads': 123, 'scurring': 124, 'skin_peeling': 125, 'silver_like_dusting': 126, 'small_dents_in_nails': 127, 'inflammatory_nails': 128, 'blister': 129, 'red_sore_around_nose': 130, 'yellow_crust_ooze': 131}
diseases_list = {15: 'Fungal infection', 4: 'Allergy', 16: 'GERD', 9: 'Chronic cholestasis',1:"AIDS",14: 'Drug Reaction', 34: 'Peptic ulcer diseae', 12: 'Diabetes ', 17: 'Gastroenteritis', 6: 'Bronchial Asthma', 23: 'Hypertension ', 31: 'Migraine', 7: 'Cervical spondylosis', 33: 'Paralysis (brain hemorrhage)', 29: 'Jaundice', 30: 'Malaria', 8: 'Chicken pox', 11: 'Dengue', 38: 'Typhoid', 41: 'hepatitis A', 19: 'Hepatitis B', 20: 'Hepatitis C', 21: 'Hepatitis D', 22: 'Hepatitis E', 3: 'Alcoholic hepatitis', 37: 'Tuberculosis', 10: 'Common Cold', 35: 'Pneumonia', 13: 'Dimorphic hemmorhoids(piles)', 39:'Urinary tract infection',18: 'Heart attack', 40: 'Varicose veins', 26: 'Hypothyroidism', 24: 'Hyperthyroidism', 25: 'Hypoglycemia', 32: 'Osteoarthristis', 5: 'Arthritis', 0: '(vertigo) Paroymsal  Positional Vertigo', 2: 'Acne', 36: 'Psoriasis', 27: 'Impetigo',28:'Influenza (Flu)'}

#Model Prediction Function
def predict_disease(patient_symptoms):
    input_vector = np.zeros(len(symptoms_dict))

    for item in patient_symptoms:
        input_vector[symptoms_dict[item]] = 1
    return diseases_list[best_svc.predict([input_vector])[0]]

In [32]:
# test 1
symptoms=input("Enter your symptoms: ")
user_symptoms=[s.strip() for s in symptoms.split(',') ]
user_symptoms=[sym.strip("[]' ") for sym in user_symptoms]
predicted_disease=predict_disease(user_symptoms)
desc,pre,med,health,diet=helper(predicted_disease)

#results print
print("======================== Predicted Disease =========================")
print(predicted_disease)
print("======================== Disease Description =======================")
print(desc)
print("======================== Disease Precaution ========================")
i=1
for p_i in pre[0]:
  print(i,": ",p_i)
  i+=1

print("======================== Disease Medication ========================")
i=1
for m_i in med:
    print(i,": ",m_i)
    i+=1

print("======================== Disease Health Advice =======================")
i=1
for h_i in health:
  print(i,": ",h_i)
  i+=1

print("========================= Disease Diet ================================")
i=1
for d_i in diet:
  print(i,": ",d_i)
  i+=1

Enter your symptoms: itching
======================== Predicted Disease =========================
Fungal infection
======================== Disease Description =======================
Fungal infection is a common skin condition caused by fungi.
======================== Disease Precaution ========================
1 :  bath twice
2 :  use detol or neem in bathing water
3 :  keep infected area dry
4 :  use clean cloths
======================== Disease Medication ========================
1 :  ['Antifungal Cream', 'Fluconazole', 'Terbinafine', 'Clotrimazole', 'Ketoconazole']
======================== Disease Health Advice =======================
1 :  Avoid sugary foods
2 :  Consume probiotics
3 :  Increase intake of garlic
4 :  Include yogurt in diet
5 :  Limit processed foods
6 :  Stay hydrated
7 :  Consume green tea
8 :  Eat foods rich in zinc
9 :  Include turmeric in diet
10 :  Eat fruits and vegetables
========================= Disease Diet ================================
1 :  ['Antifun

In [34]:
# test 2
symptoms=input("Enter your symptoms: ")
user_symptoms=[s.strip() for s in symptoms.split(',') ]
user_symptoms=[sym.strip("[]' ") for sym in user_symptoms]
predicted_disease=predict_disease(user_symptoms)
desc,pre,med,health,diet=helper(predicted_disease)

#results print
print("======================== Predicted Disease =========================")
print(predicted_disease)
print("======================== Disease Description =======================")
print(desc)
print("======================== Disease Precaution ========================")
i=1
for p_i in pre[0]:
  print(i,": ",p_i)
  i+=1

print("======================== Disease Medication ========================")
i=1
for m_i in med:
    print(i,": ",m_i)
    i+=1

print("======================== Disease Health Advice =======================")
i=1
for h_i in health:
  print(i,": ",h_i)
  i+=1

print("========================= Disease Diet ================================")
i=1
for d_i in diet:
  print(i,": ",d_i)
  i+=1

Enter your symptoms: high_fever,shivering
======================== Predicted Disease =========================
Allergy
======================== Disease Description =======================
Allergy is an immune system reaction to a substance in the environment.
======================== Disease Precaution ========================
1 :  apply calamine
2 :  cover area with bandage
3 :  nan
4 :  use ice to compress itching
======================== Disease Medication ========================
1 :  ['Antihistamines', 'Decongestants', 'Epinephrine', 'Corticosteroids', 'Immunotherapy']
======================== Disease Health Advice =======================
1 :  Avoid allergenic foods
2 :  Consume anti-inflammatory foods
3 :  Include omega-3 fatty acids
4 :  Stay hydrated
5 :  Eat foods rich in vitamin C
6 :  Include quercetin-rich foods
7 :  Consume local honey
8 :  Limit processed foods
9 :  Include ginger in diet
10 :  Avoid artificial additives
========================= Disease Diet ============

In [36]:
# test 3
symptoms=input("Enter your symptoms: ")
user_symptoms=[s.strip() for s in symptoms.split(',') ]
user_symptoms=[sym.strip("[]' ") for sym in user_symptoms]
predicted_disease=predict_disease(user_symptoms)
desc,pre,med,health,diet=helper(predicted_disease)

#results print
print("======================== Predicted Disease =========================")
print(predicted_disease)
print("======================== Disease Description =======================")
print(desc)
print("======================== Disease Precaution ========================")
i=1
for p_i in pre[0]:
  print(i,": ",p_i)
  i+=1

print("======================== Disease Medication ========================")
i=1
for m_i in med:
    print(i,": ",m_i)
    i+=1

print("======================== Disease Health Advice =======================")
i=1
for h_i in health:
  print(i,": ",h_i)
  i+=1

print("========================= Disease Diet ================================")
i=1
for d_i in diet:
  print(i,": ",d_i)
  i+=1

Enter your symptoms: joint_pain,hip_joint_pain
======================== Predicted Disease =========================
Osteoarthristis
======================== Disease Description =======================
Osteoarthristis is a degenerative joint disease that affects the cartilage in joints.
======================== Disease Precaution ========================
1 :  acetaminophen
2 :  consult nearest hospital
3 :  follow up
4 :  salt baths
======================== Disease Medication ========================
1 :  ['NSAIDs', 'Disease-modifying antirheumatic drugs (DMARDs)', 'Biologics', 'Corticosteroids', 'Joint replacement surgery']
======================== Disease Health Advice =======================
1 :  Consume anti-inflammatory foods
2 :  Include omega-3 fatty acids
3 :  Consume antioxidant-rich foods
4 :  Stay hydrated
5 :  Limit processed foods
6 :  Include vitamin K-rich foods
7 :  Consult a healthcare professional
8 :  Follow medical recommendations
9 :  Engage in low-impact exercise
1